# from saved

In [2]:
from sklearn.metrics import classification_report

In [3]:
# md = "microsoft_deberta-base_ep5_wus1000_lr1e-06"

md = "microsoft_deberta-large_ep5_wus100_lr1e-05"

In [4]:
import json
with open(f"GYAFC_test_labels.json") as f:
    labels_saved = json.load(f)

In [5]:
import json
with open(f"../trained_models/{md}/test_predicts.json") as f:
    list_predited_label_saved = json.load(f)

In [6]:
print(classification_report(labels_saved, list_predited_label_saved, digits=3))

              precision    recall  f1-score   support

           0      0.824     0.958     0.886     22151
           1      0.941     0.767     0.845     19449

    accuracy                          0.869     41600
   macro avg      0.883     0.862     0.866     41600
weighted avg      0.879     0.869     0.867     41600



In [23]:
print(classification_report(list_label, list_predited_label_saved, digits=3))

              precision    recall  f1-score   support

           0      0.824     0.958     0.886     22151
           1      0.941     0.767     0.845     19449

    accuracy                          0.869     41600
   macro avg      0.883     0.862     0.866     41600
weighted avg      0.879     0.869     0.867     41600



# From relaunched inference

In [8]:
import sys
sys.path.append("../")

In [9]:
from utils import train_nli
from data import load_gyafc
from torch.utils.data import Dataset, DataLoader
import torch
from transformers import (DebertaTokenizer,
                          DebertaForSequenceClassification,
                          AutoModelForSequenceClassification,  
                          AutoTokenizer,
                          Trainer,TrainingArguments)

2023-05-05 14:29:01.992502: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-05 14:29:02.674039: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
tr_val_test_datasets = load_gyafc("microsoft/deberta-base", toy = False)

In [12]:
_, _, test_dataset = tr_val_test_datasets

In [13]:
test_dataloader = DataLoader(
            test_dataset,
            batch_size=64,
            # num_workers=4,
            drop_last=True
        )

In [14]:
trained_model = f"../trained_models/{md}/nli_model/"

In [15]:
model = AutoModelForSequenceClassification.from_pretrained(trained_model)
model = model.to(device)

In [16]:
list_predited_label = []
list_label = []
with torch.no_grad():
    for d in test_dataloader:
        input_ids = d["input_ids"].to(device) # .reshape(64, 24)
        attention_mask = d["attention_mask"].to(device)

        outputs = model(input_ids, attention_mask)
        logits = outputs[0]

        _, prediction = torch.max(logits, dim=1)
        targets = d["labels"].detach().numpy().tolist()
        prediction = prediction.cpu().detach().numpy().tolist()

        list_label.extend(targets)
        list_predited_label.extend(prediction)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [19]:
print(classification_report(labels_saved, list_predited_label, digits=3))

              precision    recall  f1-score   support

           0      0.824     0.958     0.886     22151
           1      0.941     0.767     0.845     19449

    accuracy                          0.869     41600
   macro avg      0.883     0.862     0.866     41600
weighted avg      0.879     0.869     0.867     41600



In [20]:
print(classification_report(list_label, list_predited_label, digits=3))

              precision    recall  f1-score   support

           0      0.824     0.958     0.886     22151
           1      0.941     0.767     0.845     19449

    accuracy                          0.869     41600
   macro avg      0.883     0.862     0.866     41600
weighted avg      0.879     0.869     0.867     41600



In [ ]:
              precision    recall  f1-score   support

           0      0.824     0.958     0.886     22151
           1      0.941     0.767     0.845     19449

    accuracy                          0.869     41600
   macro avg      0.883     0.862     0.866     41600
weighted avg      0.879     0.869     0.867     41600

# Sanity check

In [21]:
from collections import Counter
from sklearn.metrics import accuracy_score

In [22]:
check_stupid_labels = [a==b for a,b in zip(list_predited_label_saved, list_predited_label)]
Counter(check_stupid_labels)

Counter({True: 33001, False: 8599})

In [28]:
tst_cnt = 30

In [29]:
accuracy_score(labels_saved[:tst_cnt], list_predited_label_saved[:tst_cnt])

0.9666666666666667

In [30]:
accuracy_score(labels_saved[:tst_cnt], list_predited_label[:tst_cnt])

0.9666666666666667

In [31]:
Counter([a==b for a,b in zip(list_predited_label_saved[:tst_cnt], list_predited_label[:tst_cnt])])

Counter({True: 28, False: 2})

In [32]:
for true, pr_saved, pr_inferred in zip(labels_saved[:tst_cnt],  list_predited_label_saved[:tst_cnt], list_predited_label[:tst_cnt]):
    print(true, pr_saved, pr_inferred)

0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 1 0
0 0 0
0 0 0
0 0 1
0 0 0
0 0 0


In [58]:
list_predited_label_saved==list_predited_label

False

In [71]:
# ind = 0
# for a,b in zip(list_predited_label, list_predited_label_saved):
#     if a!=b:
#         print(a,b, ind)
        
#     ind += 1    

In [59]:
Counter([a==b for a,b in zip(labels_saved, list_label)])

Counter({True: 41600})